# TEST

[Lambda function (PDF 2 PNG)]("https://github.com/assansanogo/b_s_processor/blob/main/lambda_functions/convert_from_pdf_to_png_5_banks.py")

This notebook runs the major function of the lambda function responsible of generating the png.
Before testing the function was operating but was not extracting the right info
corrected on *02/11/2021*

In [31]:
rm -rf /tmp

In [43]:
import PyPDF2
import tqdm
from pdf2image import convert_from_path, convert_from_bytes
from zipfile import ZipFile
import os
import glob2
import requests
import json
import boto3


def download_url(url):
    '''
    utility funcction which downloads pdf to local environment
    '''
    # data is going to be read as stream
    chunk_size=2000
    r = requests.get(url, stream=True)
    # the pdf filename is extracted from the presigned url
    file_name = [el for el in url.split("/") if ".pdf" in el][0]
    # open a file to dump the stream in
    with open(f'/tmp/{file_name}', 'wb') as fd:
        for chunk in r.iter_content(chunk_size):
            fd.write(chunk)
    return f'/tmp/{file_name}'

def parse(my_pdf):
    reader = PyPDF2.PdfFileReader(my_pdf)
    n_pages = len(reader.pages)
    new_dir = '/tmp'
    os.makedirs(new_dir, exist_ok=True)
 
    
    for i in tqdm.tqdm(range(n_pages)):
        writer = PyPDF2.PdfFileWriter()
        my_page = reader.getPage(i)
        writer.addPage(my_page)
        output_filename = my_pdf.replace('.pdf', f'_{str(i)}.pdf')
        _filename = output_filename.split("/")[-1]

        new_path = os.path.join(new_dir, _filename)
        print(new_dir)
        
        with open(new_path, 'wb') as output:
            writer.write(output)
        png_path = new_path.replace("pdf","png")
        print(png_path)
        img_test = convert_from_path(new_path)[0].save(png_path)
        
           
    all_png = glob2.glob(f"{new_dir}/*.png")
    for png_path in all_png:
        with ZipFile('my_bank_statement_png.zip','a') as zip:
            # writing each file one by one for file in png paths:
            zip.write(png_path)
    print(os.path.getsize('my_bank_statement_png.zip'))
    return 'my_bank_statement_png.zip'

In [44]:
# TEST with UBA_BANK

The warning message is <u>just</u> a warning <br>(it should not happen with the lambda function as /tmp is flushed after each file)

In [45]:
parse("/Users/assansanogo/Downloads/Liberta_Leasing_5_banks/UBA_BANK/erario statement.pdf")

  0%|          | 0/16 [00:00<?, ?it/s]

/tmp
/tmp/erario statement_0.png


  6%|▋         | 1/16 [00:00<00:07,  1.94it/s]

/tmp
/tmp/erario statement_1.png


 12%|█▎        | 2/16 [00:00<00:07,  1.98it/s]

...

 94%|█████████▍| 15/16 [00:06<00:00,  2.27it/s]

/tmp
/tmp/erario statement_15.png


100%|██████████| 16/16 [00:07<00:00,  2.23it/s]


33278665


'my_bank_statement_png.zip'

In [46]:
# TEST with STANDARD_CHARTERED_BANK

In [47]:
parse("Liberta_Leasing_5_banks/STANDARD_CHARTERED_BANK/Statemen of Account USD May 2021  Adekunle Lateef Ali.pdf")

  0%|          | 0/1 [00:00<?, ?it/s]

/tmp
/tmp/Statemen of Account USD May 2021  Adekunle Lateef Ali_0.png


100%|██████████| 1/1 [00:00<00:00,  3.56it/s]


66557308


'my_bank_statement_png.zip'

In [48]:
# TEST with POLARIS_BANK

In [49]:
parse("Liberta_Leasing_5_banks/POLARIS_BANK/MR ADOLPHUS STATEMENT.pdf")

  0%|          | 0/4 [00:00<?, ?it/s]

/tmp
/tmp/MR ADOLPHUS STATEMENT_0.png


 25%|██▌       | 1/4 [00:00<00:01,  2.50it/s]

/tmp
/tmp/MR ADOLPHUS STATEMENT_1.png


 50%|█████     | 2/4 [00:00<00:00,  2.55it/s]

...

 75%|███████▌  | 3/4 [00:01<00:00,  2.58it/s]

/tmp
/tmp/MR ADOLPHUS STATEMENT_3.png


100%|██████████| 4/4 [00:01<00:00,  2.76it/s]


99835951


'my_bank_statement_png.zip'

In [50]:
# TEST with ACCESS_BANK

In [51]:
parse("Liberta_Leasing_5_banks/ACCESS_BANK/MR AUSTINE OKENU BANK STATEMENT.pdf")

  0%|          | 0/28 [00:00<?, ?it/s]

/tmp
/tmp/MR AUSTINE OKENU BANK STATEMENT_0.png


  4%|▎         | 1/28 [00:00<00:08,  3.31it/s]

/tmp
/tmp/MR AUSTINE OKENU BANK STATEMENT_1.png


  7%|▋         | 2/28 [00:00<00:07,  3.34it/s]

...

 96%|█████████▋| 27/28 [00:08<00:00,  3.38it/s]

/tmp
/tmp/MR AUSTINE OKENU BANK STATEMENT_27.png


100%|██████████| 28/28 [00:08<00:00,  3.34it/s]


133114594


'my_bank_statement_png.zip'

In [52]:
# TEST with WEMA_BANK

In [53]:
parse("Liberta_Leasing_5_banks/WEMA_BANK/MR IBRAHEEM LAWAL STATEMENT.pdf")

  0%|          | 0/23 [00:00<?, ?it/s]

/tmp
/tmp/MR IBRAHEEM LAWAL STATEMENT_0.png


  4%|▍         | 1/23 [00:00<00:07,  3.14it/s]

/tmp
/tmp/MR IBRAHEEM LAWAL STATEMENT_1.png


  9%|▊         | 2/23 [00:00<00:06,  3.09it/s]

...

 96%|█████████▌| 22/23 [00:10<00:00,  1.45it/s]

/tmp
/tmp/MR IBRAHEEM LAWAL STATEMENT_22.png


100%|██████████| 23/23 [00:11<00:00,  2.01it/s]


166393237


'my_bank_statement_png.zip'